In [1]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister

import math

# the following function builds a gate (oracle) that transforms |x>|y> to |x>|y - cut(x)>
def energy_oracle(graph_laplacian: list[list[int]], digits: int):

    # setting up the circuit
    qregx = QuantumRegister(len(graph_laplacian),"q-x")
    qregy = QuantumRegister(digits,"q-y")
    qc = QuantumCircuit(qregx,qregy)

    # QFT
    for i in range(digits):
        qc.h(qregy[i])

        theta = math.pi/2
        for j in range(i + 1, digits):
            qc.cp(theta, qregy[j], qregy[i])
            theta /= 2

    for i in range(digits // 2):
        qc.swap(qregy[i],qregy[- (i + 1)])

    # Phasers
    for i in range(len(graph_laplacian)):
        theta = - graph_laplacian[i][i] * math.pi
        for j in range(digits):
            qc.cp(theta, qregx[i], qregy[j])
            theta /= 2

        for j in range(i + 1, len(graph_laplacian[i])):
            if graph_laplacian[i][j] != 0:
                theta = - graph_laplacian[i][j] * math.pi * 2
                for k in range(digits):
                    qc.mcp(theta, [qregx[i], qregx[j]], qregy[k])
                    theta /= 2

    # Inverse QFT
    for i in range(digits // 2):
        qc.swap(qregy[i],qregy[- (i + 1)])

    for i in range(digits - 1, - 1, - 1):
        qc.h(qregy[i])

        theta = - math.pi/2
        for j in range(i - 1, - 1, - 1):
            qc.cp(theta, qregy[j], qregy[i])
            theta /= 2

    return qc.to_gate()

def grover_bangbang(graph_laplacian: list[list[int]], digits: int, alpha: float, beta: float):
    
    U_f = energy_oracle(graph_laplacian, digits)
    U_f.label = "U_f"
    U_f_inv = U_f.inverse()
    U_f_inv.label = "U_f_inv"

    qregx = QuantumRegister(len(graph_laplacian), "q-x")
    qregy = QuantumRegister(digits, "q-y")

    grover_bangbang = QuantumCircuit(qregx,qregy)

    grover_bangbang.append(U_f, qregx[:] + qregy[:])

    grover_bangbang.p(beta, qregy[0])

    grover_bangbang.append(U_f_inv, qregx[:] + qregy[:])

    for i in range(len(graph_laplacian)):
        grover_bangbang.x(i)

    grover_bangbang.mcp(alpha, qregx, qregy[0])

    for i in range(len(graph_laplacian)):
        grover_bangbang.x(i)

    print(grover_bangbang.draw())

    return grover_bangbang.to_gate()

def grover_fixed_point_search(graph_laplacian: list[list[int]], digits: int, l: int, delta: float):
    alpha = []
    for j in range(l):
        gamma = 1 / (math.cosh(math.acosh(1 / delta) / (2 * l + 1)))
        arg = math.tan(math.pi * (j + 1) / (2 * l + 1)) * math.sqrt(1 - gamma * gamma)
        alpha.append(2 * math.atan(1/arg))

    qregx = QuantumRegister(len(graph_laplacian), "q-x")
    qregy = QuantumRegister(digits, "q-y")

    gfps = QuantumCircuit(qregx,qregy)

    for i in range(len(graph_laplacian)):
        gfps.h(qregx[i])

    for j in range(l):
        G_j = grover_bangbang(graph_laplacian, digits, alpha[j], - alpha[l - j - 1])
        G_j.label = "G(alpha_" + str(j + 1) + ", beta_" + str(j + 2) + ")"
        gfps.append(G_j, qregx[:] + qregy[:])

    print(gfps)

    return gfps.to_gate()


graph_laplacian = [[3, - 1, - 1, - 1], [- 1, 1, 0, 0], [- 1, 0, 1, 0], [- 1, 0, 0, 1]]
digits = 4
alpha = 3
beta = math.pi

grover_fixed_point_search(graph_laplacian, digits, 2, 0.1)

#grover_bangbang(graph_laplacian, digits, alpha, beta)
#energy_oracle(graph_laplacian, digits)

       ┌──────┐              ┌──────────┐┌───┐            ┌───┐
q-x_0: ┤0     ├──────────────┤0         ├┤ X ├─■──────────┤ X ├
       │      │              │          │├───┤ │          ├───┤
q-x_1: ┤1     ├──────────────┤1         ├┤ X ├─■──────────┤ X ├
       │      │              │          │├───┤ │          ├───┤
q-x_2: ┤2     ├──────────────┤2         ├┤ X ├─■──────────┤ X ├
       │      │              │          │├───┤ │          ├───┤
q-x_3: ┤3     ├──────────────┤3         ├┤ X ├─■──────────┤ X ├
       │  U_f │┌────────────┐│  U_f_inv │└───┘ │P(2.3988) └───┘
q-y_0: ┤4     ├┤ P(-1.0898) ├┤4         ├──────■───────────────
       │      │└────────────┘│          │                      
q-y_1: ┤5     ├──────────────┤5         ├──────────────────────
       │      │              │          │                      
q-y_2: ┤6     ├──────────────┤6         ├──────────────────────
       │      │              │          │                      
q-y_3: ┤7     ├──────────────┤7         

Instruction(name='circuit-114', num_qubits=8, num_clbits=0, params=[])